### **Group 8**

In [29]:
#importing relevant libraries
from google.colab import drive
import pandas as pd
import numpy as np
import re

In [30]:
#mounting to google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
#file paths to all the datasets
facility_path = '/content/drive/My Drive/HTC_Case/Facility_Data Pull.csv'
geoscout_path = '/content/drive/My Drive/HTC_Case/Geoscout Facility Data Dump.csv'
emissions_path = '/content/drive/My Drive/HTC_Case/PDGES-GHGRP-GHGEmissionsGES-2004-Present.xlsx'

In [32]:
#reading files to a pandas dataframe
facility_df = pd.read_csv(facility_path)
geoscout_df = pd.read_csv(geoscout_path)
emissions_df = pd.read_excel(emissions_path)

<ipython-input-32-a103d5bf5927>:2: DtypeWarning: Columns (4,7,8,45) have mixed types. Specify dtype option on import or set low_memory=False.
  facility_df = pd.read_csv(facility_path)
<ipython-input-32-a103d5bf5927>:3: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  geoscout_df = pd.read_csv(geoscout_path)


Examining the facility_df dataframe

In [33]:
facility_df.head()

,Unique Facility ID,Status,Name,Type,Plant Name,Current Operator,Original Operator,Co-Owner,Constructed Date,Activated Date,...,Gas Flared(Mcf/d).1,Gas Vented(Mcf/d).1,Gas Meter Diff(Mcf/d).1,Gas Delivered(Mcf/d).1,Water Production(Bbl/d).1,Water Received(Bbl/d).1,Water Open Inventory(Bbl/d).1,Water Close Inventory(Bbl/d).1,Water Meter Diff(Bbl/d).1,Water Delivered(Bbl/d).1
0,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,MAPLE GLEN,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,MAPLE GLEN,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,MAPLE GLEN,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,MAPLE GLEN,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,MAPLE GLEN,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape of the facility_df dataframe

In [34]:
facility_df.shape

(244139, 127)

All the columns in the facility_df dataframe

In [35]:
for col in facility_df.columns:
    print(col)

Unique Facility ID
Status
Name
Type
Plant Name
Current Operator
Original Operator
Co-Owner
Constructed Date
Activated Date
Field Name
Latitude
Longitude
Licensed Capacity of Raw Gas(Mcf/d)
Licensed Capacity of Sales Gas(Mcf/d)
Licensed Capacity of Oil(Bbl/d)
Licensed Capacity of Ethane (C2)(Bbl/d)
Licensed Capacity of Ethane Plus (C2+)(Bbl/d)
Licensed Capacity of Propane (C3)(Bbl/d)
Licensed Capacity of Butane (C4)(Bbl/d)
Licensed Capacity of Pentane Plus (C5+)(Bbl/d)
Licensed Capacity of NGL(Bbl/d)
Licensed Capacity of LPG(Bbl/d)
Licensed Capacity of Sulphur(ton/d)
Design Capacity of Raw Gas(Mcf/d)
Design Capacity of Sales Gas(Mcf/d)
Design Capacity of Oil(Bbl/d)
Design Capacity of Ethane (C2)(Bbl/d)
Design Capacity of Ethane Plus (C2+)(Bbl/d)
Design Capacity of Propane (C3)(Bbl/d)
Design Capacity of Butane (C4)(Bbl/d)
Design Capacity of Pentane Plus (C5+)(Bbl/d)
Design Capacity of NGL(Bbl/d)
Design Capacity of LPG(Bbl/d)
Design Capacity of Sulphur(ton/d)
Raw Gas(Mcf/d)
Sales Gas(Mcf/

Checking for missing values in the facility_df dataframe

In [36]:
facility_df.isnull().sum()

,0
Unique Facility ID,0
Status,0
Name,137
Type,0
Plant Name,15450
...,...
Water Received(Bbl/d).1,244139
Water Open Inventory(Bbl/d).1,244139
Water Close Inventory(Bbl/d).1,244139
Water Meter Diff(Bbl/d).1,244139


We observe that some columns have all their rows missing. These columns can be dropped directly.

In [37]:
missing_counts = facility_df.isnull().sum()
fully_missing_cols = missing_counts[missing_counts == facility_df.shape[0]]

print(f"Total number of fully missing columns: {len(fully_missing_cols)}")

for col in fully_missing_cols.index:
    print(col)

Total number of fully missing columns: 54
Original Operator
Activated Date
Field Name
Licensed Capacity of Oil(Bbl/d)
Licensed Capacity of NGL(Bbl/d)
Design Capacity of Ethane Plus (C2+)(Bbl/d)
Design Capacity of LPG(Bbl/d)
Liquefied Petroleum Gas(Bbl/d)
Oil(Bbl/d)
Methane Plus (C1+)(Bbl/d)
Receipt of Ethane (C2)(Bbl/d)
Receipt of Ethane Plus (C2+)(Bbl/d)
Receipt of Propane (C3)(Bbl/d)
Receipt of Propane Plus (C3+)(Bbl/d)
Receipt of Butane (C4)(Bbl/d)
Receipt of Butane Plus (C4+)(Bbl/d)
Receipt of Pentane (C5)(Bbl/d)
Receipt of Pentane Plus (C5+)(Bbl/d)
Oil Production(Bbl/d)
Oil Received(Bbl/d)
Oil Open Inventory(Bbl/d)
Oil Close Inventory(Bbl/d)
Oil Delivered(Bbl/d)
Gas Production(Mcf/d)
Gas Received(Mcf/d)
Lease Fuel(Mcf/d)
Gas Flared(Mcf/d)
Gas Vented(Mcf/d)
Gas Meter Diff(Mcf/d)
Gas Delivered(Mcf/d)
Water Production(Bbl/d)
Water Received(Bbl/d)
Water Open Inventory(Bbl/d)
Water Close Inventory(Bbl/d)
Water Meter Diff(Bbl/d)
Water Delivered(Bbl/d)
Oil Production(Bbl/d).1
Oil Receive

Total = 127 columns

Total number of fully missing columns: 54

127-54 = 73 columns should remain

In [38]:
facility_df = facility_df.drop(columns=fully_missing_cols.index)
facility_df.shape

(244139, 73)

Let's check for any repeated columns in the facility_df dataframe

In [39]:
for col in facility_df.columns:
    print(col)

Unique Facility ID
Status
Name
Type
Plant Name
Current Operator
Co-Owner
Constructed Date
Latitude
Longitude
Licensed Capacity of Raw Gas(Mcf/d)
Licensed Capacity of Sales Gas(Mcf/d)
Licensed Capacity of Ethane (C2)(Bbl/d)
Licensed Capacity of Ethane Plus (C2+)(Bbl/d)
Licensed Capacity of Propane (C3)(Bbl/d)
Licensed Capacity of Butane (C4)(Bbl/d)
Licensed Capacity of Pentane Plus (C5+)(Bbl/d)
Licensed Capacity of LPG(Bbl/d)
Licensed Capacity of Sulphur(ton/d)
Design Capacity of Raw Gas(Mcf/d)
Design Capacity of Sales Gas(Mcf/d)
Design Capacity of Oil(Bbl/d)
Design Capacity of Ethane (C2)(Bbl/d)
Design Capacity of Propane (C3)(Bbl/d)
Design Capacity of Butane (C4)(Bbl/d)
Design Capacity of Pentane Plus (C5+)(Bbl/d)
Design Capacity of NGL(Bbl/d)
Design Capacity of Sulphur(ton/d)
Raw Gas(Mcf/d)
Sales Gas(Mcf/d)
Ethane (C2)(Bbl/d)
Ethane Plus (C2+)(Bbl/d)
Propane (C3)(Bbl/d)
Butane (C4)(Bbl/d)
Pentane Plus (C5+)(Bbl/d)
Natural Gas Liquids(Bbl/d)
Sulphur(ton/d)
Sub Type
Location
Injection(

We also observe that there might be certain repeated columns in our dataset. Let's see whether those are just the column names or whether their rows have the same values corresponding values or not.

In [40]:
original_cols = [col for col in facility_df.columns if not col.endswith('.1')]
duplicate_cols = [col + '.1' for col in original_cols if col + '.1' in facility_df.columns]

duplicate_check = {}

for orig, dup in zip(original_cols, duplicate_cols):
    duplicate_check[orig] = facility_df[orig].equals(facility_df[dup])

for key, value in duplicate_check.items():
    if value:
        print(f"Columns '{key}' and '{key}.1' are identical.")
    else:
        print(f"Columns '{key}' and '{key}.1' contain different values.")

Columns 'Unique Facility ID' and 'Unique Facility ID.1' contain different values.
Columns 'Status' and 'Status.1' contain different values.
Columns 'Name' and 'Name.1' contain different values.
Columns 'Type' and 'Type.1' contain different values.
Columns 'Plant Name' and 'Plant Name.1' contain different values.
Columns 'Current Operator' and 'Current Operator.1' contain different values.
Columns 'Co-Owner' and 'Co-Owner.1' contain different values.
Columns 'Constructed Date' and 'Constructed Date.1' contain different values.
Columns 'Latitude' and 'Latitude.1' contain different values.
Columns 'Longitude' and 'Longitude.1' contain different values.
Columns 'Licensed Capacity of Raw Gas(Mcf/d)' and 'Licensed Capacity of Raw Gas(Mcf/d).1' contain different values.
Columns 'Licensed Capacity of Sales Gas(Mcf/d)' and 'Licensed Capacity of Sales Gas(Mcf/d).1' contain different values.
Columns 'Licensed Capacity of Ethane (C2)(Bbl/d)' and 'Licensed Capacity of Ethane (C2)(Bbl/d).1' contain 

In [41]:
duplicate_columns = facility_df.columns[facility_df.columns.duplicated()]
print(duplicate_columns)

Index([], dtype='object')


We do not have any duplicate columns in our facility_df. Although some columns have a '.1' suffix, they represent distinct data with differing row values compared to their corresponding columns.

Now let's look into some important columns with missing values that could be beneficial to dive into for our analysis.

* Longitude
* Latitude
* Name
* Type
* Plant Name
* Current Operator
* Location
* Sub Type
* Status
* Co-Owner

In [42]:
columns_to_check = ['Longitude', 'Latitude', 'Name', 'Type', 'Plant Name', 'Current Operator', 'Location', 'Sub Type', 'Status', 'Co-Owner']

In [43]:
for col in columns_to_check:
    print(f"Unique values in {col}:")
    print(facility_df[col].unique())
    print("\n")

Unique values in Longitude:
['112.15592 deg W' '113.07704 deg W' '113.79413 deg W' ...
 '120.82323 deg W' '121.29843 deg W' nan]


Unique values in Latitude:
['52.14129 deg N' '52.31647 deg N' '49.30742 deg N' ... '56.02313 deg N'
 '56.05613 deg N' nan]


Unique values in Name:
['MAPLE GLEN 15-36-036-16W4' 'KEYERA NEVIS GAS PLANT 16-33-38-22W4'
 '* GULF PINCHER CREEK' ...
 'NORTHRIVER OPSGP DAWSONCREEK 11-26-078-17 003'
 'CNRL SEPTIMUS 09-13-081-19 001' 'CREW MONIAS 15-25-081-22 001']


Unique values in Type:
['GP']


Unique values in Plant Name:
['MAPLE GLEN' 'NEVIS' 'PINCHER CREEK' 'RIMBEY' 'GILBY' 'MITSUE' 'PRINCESS'
 'SEDALIA' 'CAMPBELL' 'INNISFAIL' 'MAKEPEACE' 'CARBON' 'WARDLOW'
 'COMETRA WOOD RIVER' 'CESSFORD' 'PROVOST' 'BUCK CREEK' 'CARSTAIRS'
 'GARRINGTON' 'LEDUC WOODBEND' 'BOUNDARY LAKE SOUTH' 'NORMANDVILLE'
 'GOLDEN SPIKE' 'REDWATER' 'FERRYBANK' 'TURNER VALLEY' 'VALHALLA'
 'WORSLEY' 'THREE HILLS CREEK' 'WINDFALL' 'ICG ACHESON' 'CONSORT'
 'JUMPING POUND' 'COUNTESS' 'ENCHANT' '

In [44]:
for col in columns_to_check:
    missing_count = facility_df[col].isnull().sum()
    print(f"Missing values in {col}: {missing_count}")

Missing values in Longitude: 2
Missing values in Latitude: 2
Missing values in Name: 137
Missing values in Type: 0
Missing values in Plant Name: 15450
Missing values in Current Operator: 678
Missing values in Location: 2
Missing values in Sub Type: 13067
Missing values in Status: 0
Missing values in Co-Owner: 21440


Latitude and Longitude column cleaning

In [45]:
facility_df['Longitude'].unique()

array(['112.15592 deg W', '113.07704 deg W', '113.79413 deg W', ...,
       '120.82323 deg W', '121.29843 deg W', nan], dtype=object)

In [46]:
facility_df['Latitude'].unique()

array(['52.14129 deg N', '52.31647 deg N', '49.30742 deg N', ...,
       '56.02313 deg N', '56.05613 deg N', nan], dtype=object)

Let's clean these columns by extracting the numeric part of the Longitude and Latitude columns, removing the direction (e.g., 'deg N', 'deg S', 'deg E', 'deg W'), so that we are left with only the numeric values.

In [47]:
facility_df['Longitude'] = facility_df['Longitude'].str.extract(r'([\-+]?\d*\.\d+)').astype(float)
facility_df['Latitude'] = facility_df['Latitude'].str.extract(r'([\-+]?\d*\.\d+)').astype(float)

facility_df[['Longitude', 'Latitude']]

,Longitude,Latitude
0,112.15592,52.14129
1,112.15592,52.14129
2,112.15592,52.14129
3,112.15592,52.14129
4,112.15592,52.14129
...,...,...
244134,120.52433,55.79354
244135,120.82323,56.02313
244136,121.29843,56.05613
244137,NaN,NaN


In [48]:
facility_df['Longitude'] = - facility_df['Longitude']

Conversion for Longitude from East to West because the coordinates were reversed.

Creating a new column by the name of Interest. The Interest column holds the value of the Interest % for the current operator.

In [49]:
facility_df['Current Operator'].unique()

array(['CANADIAN NATURAL RESOURCES LIMITED', 'KEYERA ENERGY LTD.',
       'GULF CANADA LIMITED', 'I3 ENERGY CANADA LTD.',
       'CHEVRON CANADA LIMITED', 'BAYTEX ENERGY LTD.',
       'SHELL CANADA LIMITED', 'TORXEN ENERGY LTD.',
       'ATCO NEXT ENERGY LTD.', 'VINTAGE PETROLEUM CANADA; INC.',
       'TAMARACK VALLEY ENERGY LTD.', 'BP CANADA ENERGY GROUP ULC',
       'HWN ENERGY LTD.', 'WHITECAP RESOURCES INC.',
       'IMPERIAL OIL RESOURCES LIMITED', 'CONIFER ENERGY INC.',
       'OBSIDIAN ENERGY LTD.', 'REPSOL OIL & GAS CANADA INC.',
       'PINE CLIFF ENERGY LTD.', 'PEMBINA GAS INFRASTRUCTURE INC.',
       'CNOOC PETROLEUM NORTH AMERICA ULC', 'KARVE ENERGY INC.',
       'PIERIDAE ALBERTA PRODUCTION LTD.', 'JOURNEY ENERGY INC.',
       'CAMPUS ENERGY PARTNERS OPERATIONS INC.', 'CENOVUS ENERGY INC.',
       'NUVISTA ENERGY LTD.', 'CANLIN RESOURCES PARTNERSHIP',
       'OVINTIV CANADA ULC', 'PARAMOUNT RESOURCES LTD.', 'ALTAGAS LTD.',
       'BATTLE RIVER ENERGY LTD.', 'BARREL OIL COR

In [50]:
facility_df['Co-Owner'].unique()

array(['0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADIAN NATURAL RESOURCES LIMITED',
       '100.0% - KEYERA ENERGY LTD.', '100.0% - PALMER RANCH (1984) LTD.',
       '0.0% - CENTIPEDE HOLDINGS LTD.; 0.8% - BELLATRIX EXPLORATION LTD.; 0.1% - ALTAGAS PIPELINE PARTNERSHIP; 98.8% - KEYERA ENERGY LTD.; 0.1% - MOSAIC ENERGY LTD.; 0.0% - CANADIAN NATURAL RESOURCES LIMITED; 0.0% - BONAVISTA ENERGY CORPORATION; 1.0% - GREEN VALLEY ENERGY LTD.',
       '100.0% - I3 ENERGY CANADA LTD.',
       '100.0% - CHEVRON CANADA LIMITED',
       '90.0% - CANADIAN NATURAL RESOURCES LIMITED; 10.0% - SUPERMAN RESOURCES INC.',
       '100.0% - BAYTEX ENERGY LTD.',
       '100.0% - CANADIAN NATURAL RESOURCES LIMITED',
       '100.0% - ENHANCE ENERGY INC.', '100.0% - TORXEN ENERGY LTD.',
       '80.2% - KEYERA ENERGY LTD.; 1.2% - HARVEST OPERATIONS CORP.; 5.1% - PENGROWTH ENERGY CORPORATION; 5.1% - NAL RESOURCES LIMITED; 5.7% - OMERS ENERGY INC.; 1.3% - SIGNALTA RESOURCES LIMITED',
       '100.0% - ATCO NEXT ENE

Let's create a new column "Interest" by extracting the Interest percentage of the current operator using the co-owner column.

In [51]:
def extract_interest(co_owner, operator):
    if isinstance(co_owner, str):
        owners = co_owner.split(';')
    else:
        owners = []

    interest = 0.0

    for owner in owners:
        match = re.search(r'(\d+(\.\d+)?)%\s*-\s*(.*)', owner.strip())
        if match:
            percent = float(match.group(1))
            owner_name = match.group(3).strip()

            if operator and isinstance(operator, str) and operator.strip() in str(owner_name):
                interest += percent

    return interest

facility_df['interest'] = facility_df.apply(
    lambda row: extract_interest(row['Co-Owner'], row['Current Operator']), axis=1
)

In [52]:
facility_df[['Current Operator', 'Co-Owner', 'interest']]

,Current Operator,Co-Owner,interest
0,CANADIAN NATURAL RESOURCES LIMITED,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,0.0
1,CANADIAN NATURAL RESOURCES LIMITED,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,0.0
2,CANADIAN NATURAL RESOURCES LIMITED,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,0.0
3,CANADIAN NATURAL RESOURCES LIMITED,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,0.0
4,CANADIAN NATURAL RESOURCES LIMITED,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,0.0
...,...,...,...
244134,NORTHRIVER MIDSTREAM OPERATIONS GP INC.,NaN,0.0
244135,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0
244136,CREW ENERGY INC.,NaN,0.0
244137,NORTHRIVER MIDSTREAM ENERGY LIMITED,NaN,0.0


In [53]:
facility_df['interest'].unique()

array([  0. , 100. ,  98.8,  90. ,  80.2,  50. ,  52. ,  53.3,  62. ,
        79.8,  46.7,  87.3,  95. ,  98. ,  51. ,  40. ,  55. ,  99.8,
        90.9,  75. ,  45.8,  96.1,  89.4,  85. ,  94.2,  82.2,  98.3,
        75.9,  82.9,  38.9,  88.3,  83.5,  80. ,  78.3,  74.9,  54. ,
        70.8,  75.5,  63. ,  43. ,  65. ,  73.9,  61.4,  89. ,  62.9,
        94.5,  96. ,  65.7,  78. ,  10.4,  19.1,  63.4,  83.3,  37.2,
        54.7,  86.5,  60. ,  98.4,  84. ,  83.1,  69.5,  68. ,  99. ,
        38.8,  55.1,  56. ,  31.8,  88.7,  70. ,  74. ,  94. ,  54.6,
        86.4,  62.7,  72. ,  85.4,  76.7,  62.5,  78.2,  81. ,  34.9,
        43.1,  76.9,  44. ,  41. ,  87.5,  76.8,  36.5,  57. ,  88.2,
        52.2,  87. ,  26.6,  89.5])

Removing trailing and leading spaces from the string columns to standardize the string data.

In [54]:
facility_df['Plant Name'] = facility_df['Plant Name'].str.strip()
facility_df['Current Operator'] = facility_df['Current Operator'].str.strip()
facility_df['Status'] = facility_df['Status'].str.strip()
facility_df['Name'] = facility_df['Name'].str.strip()

We are imputing the Name column. Since the Unique Facility ID column has no missing values and each ID uniquely corresponds to a facility name, we can use the Unique Facility ID to infer the missing facility names.

In [55]:
name_mapping = facility_df.dropna(subset=['Name'])[['Unique Facility ID', 'Name']].set_index('Unique Facility ID')['Name'].to_dict()

facility_df['Name'] = facility_df.apply(
    lambda row: name_mapping.get(row['Unique Facility ID'], row['Name']) if pd.isna(row['Name']) else row['Name'],
    axis=1
)

In [56]:
facility_df[['Unique Facility ID', 'Name']]

,Unique Facility ID,Name
0,ABGP0001001,MAPLE GLEN 15-36-036-16W4
1,ABGP0001001,MAPLE GLEN 15-36-036-16W4
2,ABGP0001001,MAPLE GLEN 15-36-036-16W4
3,ABGP0001001,MAPLE GLEN 15-36-036-16W4
4,ABGP0001001,MAPLE GLEN 15-36-036-16W4
...,...,...
244134,BCGP0026951,NORTHRIVER OPSGP DAWSONCREEK 11-26-078-17 003
244135,BCGP0027469,CNRL SEPTIMUS 09-13-081-19 001
244136,BCGP0027707,CREW MONIAS 15-25-081-22 001
244137,NEB8039,NaN


In [57]:
facility_df['Name'].isnull().sum()

137

There is a possibility that the data is incomplete because each Unique Facility ID should map to a unique facility name. However, if the facility has a non-active status, the name might not be available or recorded in the dataset, leading to missing values.

 We iterate through rows in the dataframe where either the Plant Name or Name is missing. For each missing value, it looks for other rows in the dataset that share the same Latitude and Longitude but have non-missing Plant Name and Name values. If a matching row is found, the Plant Name and Name values from that row are used to fill in the missing values in the current row.

In [58]:
for index, row in facility_df[facility_df['Plant Name'].isna() | facility_df['Name'].isna()].iterrows():
    missing_lat = row['Latitude']
    missing_lon = row['Longitude']

    match_row = facility_df[
        (facility_df['Latitude'] == missing_lat) & (facility_df['Longitude'] == missing_lon) &
        (facility_df['Plant Name'].notna()) & (facility_df['Name'].notna())
    ]

    if not match_row.empty:
        facility_df.at[index, 'Plant Name'] = match_row['Plant Name'].values[0]
        facility_df.at[index, 'Name'] = match_row['Name'].values[0]

In [59]:
facility_df[['Unique Facility ID', 'Latitude', 'Longitude', 'Plant Name', 'Name']]

,Unique Facility ID,Latitude,Longitude,Plant Name,Name
0,ABGP0001001,52.14129,-112.15592,MAPLE GLEN,MAPLE GLEN 15-36-036-16W4
1,ABGP0001001,52.14129,-112.15592,MAPLE GLEN,MAPLE GLEN 15-36-036-16W4
2,ABGP0001001,52.14129,-112.15592,MAPLE GLEN,MAPLE GLEN 15-36-036-16W4
3,ABGP0001001,52.14129,-112.15592,MAPLE GLEN,MAPLE GLEN 15-36-036-16W4
4,ABGP0001001,52.14129,-112.15592,MAPLE GLEN,MAPLE GLEN 15-36-036-16W4
...,...,...,...,...,...
244134,BCGP0026951,55.79354,-120.52433,NaN,NORTHRIVER OPSGP DAWSONCREEK 11-26-078-17 003
244135,BCGP0027469,56.02313,-120.82323,NaN,CNRL SEPTIMUS 09-13-081-19 001
244136,BCGP0027707,56.05613,-121.29843,NaN,CREW MONIAS 15-25-081-22 001
244137,NEB8039,NaN,NaN,NaN,NaN


In [60]:
print("Missing values in 'Plant Name':", facility_df['Plant Name'].isna().sum())
print("Missing values in 'Name':", facility_df['Name'].isna().sum())

Missing values in 'Plant Name': 15066
Missing values in 'Name': 111


We were able to reduce the missing values down further but not very significantly.

In [61]:
facility_df[['Name','Location']]

,Name,Location
0,MAPLE GLEN 15-36-036-16W4,15-36-036-16W4M
1,MAPLE GLEN 15-36-036-16W4,15-36-036-16W4M
2,MAPLE GLEN 15-36-036-16W4,15-36-036-16W4M
3,MAPLE GLEN 15-36-036-16W4,15-36-036-16W4M
4,MAPLE GLEN 15-36-036-16W4,15-36-036-16W4M
...,...,...
244134,NORTHRIVER OPSGP DAWSONCREEK 11-26-078-17 003,11-26-078-17W6M
244135,CNRL SEPTIMUS 09-13-081-19 001,09-13-081-19W6M
244136,CREW MONIAS 15-25-081-22 001,15-25-081-22W6M
244137,NaN,NaN


We can try to use the location variable to impute the Name but since we are not familiar with the overall naming convention we cannot use that approach for the purpose of data consistency. Same goes for reverse geocoding approaches.

Saving the dataframe to an excel file

In [62]:
facility_df.to_excel('/content/drive/My Drive/HTC_Case/facility_final_cleaned.xlsx', index=False)

Examining the geoscout_df

In [63]:
geoscout_df.head()

,Unique Facility ID,Status,Name,Type,Plant Name,Current Operator,Original Operator,Co-Owner,Constructed Date,Activated Date,...,Gas Flared(Mcf/d),Gas Vented(Mcf/d),Gas Meter Diff(Mcf/d),Gas Delivered(Mcf/d),Water Production(Bbl/d),Water Received(Bbl/d),Water Open Inventory(Bbl/d),Water Close Inventory(Bbl/d),Water Meter Diff(Bbl/d),Water Delivered(Bbl/d)
0,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,Maple Glen,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,Maple Glen,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,Maple Glen,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,Maple Glen,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABGP0001001,Active,MAPLE GLEN 15-36-036-16W4,GP,Maple Glen,CANADIAN NATURAL RESOURCES LIMITED,NaN,0.0% - CANSEARCH RESOURCES LTD.; 0.0% - CANADI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape of the geoscout_df dataframe

In [64]:
geoscout_df.shape

(223630, 94)

All columns in the geoscout_df

In [65]:
for col in geoscout_df.columns:
    print(col)

Unique Facility ID
Status
Name
Type
Plant Name
Current Operator
Original Operator
Co-Owner
Constructed Date
Activated Date
Field Name
Latitude
Longitude
Licensed Capacity of Raw Gas(Mcf/d)
Licensed Capacity of Sales Gas(Mcf/d)
Licensed Capacity of Oil(Bbl/d)
Licensed Capacity of Ethane(C2)(Bbl/d)
Licensed Capacity of Ethane Plus(C2+)(Bbl/d)
Licensed Capacity of Propane(C3)(Bbl/d)
Licensed Capacity of Butane(C4)(Bbl/d)
Licensed Capacity of Pentane Plus(C5+)(Bbl/d)
Licensed Capacity of NGL(Bbl/d)
Licensed Capacity of LPG(Bbl/d)
Licensed Capacity of Sulphur(ton/d)
Design Capacity of Raw Gas(Mcf/d)
Design Capacity of Sales Gas(Mcf/d)
Design Capacity of Oil(Bbl/d)
Design Capacity of Ethane(C2)(Bbl/d)
Design Capacity of Ethane Plus(C2+)(Bbl/d)
Design Capacity of Propane(C3)(Bbl/d)
Design Capacity of Butane(C4)(Bbl/d)
Design Capacity of Pentane Plus(C5+)(Bbl/d)
Design Capacity of NGL(Bbl/d)
Design Capacity of LPG(Bbl/d)
Design Capacity of Sulphur(ton/d)
Raw Gas(Mcf/d)
Sales Gas(Mcf/d)
Ethane(

Check for missing values in the geoscout_df dataframe

In [66]:
geoscout_df.isnull().sum()

,0
Unique Facility ID,0
Status,0
Name,135
Type,0
Plant Name,1186
...,...
Water Received(Bbl/d),223630
Water Open Inventory(Bbl/d),223630
Water Close Inventory(Bbl/d),223630
Water Meter Diff(Bbl/d),223630


We observe that some columns have all their rows missing. These columns can be dropped directly.

In [67]:
missing_counts = geoscout_df.isnull().sum()
fully_missing_cols = missing_counts[missing_counts == geoscout_df.shape[0]]

print(f"Total number of fully missing columns: {len(fully_missing_cols)}")

for col in fully_missing_cols.index:
    print(col)

Total number of fully missing columns: 25
Original Operator
Activated Date
Field Name
Licensed Capacity of Oil(Bbl/d)
Design Capacity of Ethane Plus(C2+)(Bbl/d)
Design Capacity of LPG(Bbl/d)
Liquefied Petroleum Gas(Bbl/d)
Oil Production(Bbl/d)
Oil Received(Bbl/d)
Oil Open Inventory(Bbl/d)
Oil Close Inventory(Bbl/d)
Oil Delivered(Bbl/d)
Gas Production(Mcf/d)
Gas Received(Mcf/d)
Lease Fuel(Mcf/d)
Gas Flared(Mcf/d)
Gas Vented(Mcf/d)
Gas Meter Diff(Mcf/d)
Gas Delivered(Mcf/d)
Water Production(Bbl/d)
Water Received(Bbl/d)
Water Open Inventory(Bbl/d)
Water Close Inventory(Bbl/d)
Water Meter Diff(Bbl/d)
Water Delivered(Bbl/d)


Total = 94 columns

Total number of fully missing columns: 25

94-25 = 69 columns should remain

In [68]:
geoscout_df = geoscout_df.drop(columns=fully_missing_cols.index)
geoscout_df.shape

(223630, 69)

Let's check for any repeated columns in the geoscout_df dataframe

In [69]:
for col in geoscout_df.columns:
    print(col)

Unique Facility ID
Status
Name
Type
Plant Name
Current Operator
Co-Owner
Constructed Date
Latitude
Longitude
Licensed Capacity of Raw Gas(Mcf/d)
Licensed Capacity of Sales Gas(Mcf/d)
Licensed Capacity of Ethane(C2)(Bbl/d)
Licensed Capacity of Ethane Plus(C2+)(Bbl/d)
Licensed Capacity of Propane(C3)(Bbl/d)
Licensed Capacity of Butane(C4)(Bbl/d)
Licensed Capacity of Pentane Plus(C5+)(Bbl/d)
Licensed Capacity of NGL(Bbl/d)
Licensed Capacity of LPG(Bbl/d)
Licensed Capacity of Sulphur(ton/d)
Design Capacity of Raw Gas(Mcf/d)
Design Capacity of Sales Gas(Mcf/d)
Design Capacity of Oil(Bbl/d)
Design Capacity of Ethane(C2)(Bbl/d)
Design Capacity of Propane(C3)(Bbl/d)
Design Capacity of Butane(C4)(Bbl/d)
Design Capacity of Pentane Plus(C5+)(Bbl/d)
Design Capacity of NGL(Bbl/d)
Design Capacity of Sulphur(ton/d)
Raw Gas(Mcf/d)
Sales Gas(Mcf/d)
Ethane(C2)(Bbl/d)
Ethane Plus(C2+)(Bbl/d)
Propane(C3)(Bbl/d)
Butane(C4)(Bbl/d)
Pentane Plus(C5+)(Bbl/d)
Natural Gas Liquids(Bbl/d)
Sulphur(ton/d)
Sub Type
L

We also observe that there might be certain repeated columns in our dataset. Let's see whether those are just the column names or whether their rows have the same values corresponding values or not.

In [70]:
original_cols = [col for col in geoscout_df.columns if not col.endswith('.1')]
duplicate_cols = [col + '.1' for col in original_cols if col + '.1' in geoscout_df.columns]

duplicate_check = {}

for orig, dup in zip(original_cols, duplicate_cols):
    duplicate_check[orig] = geoscout_df[orig].equals(geoscout_df[dup])

for key, value in duplicate_check.items():
    if value:
        print(f"Columns '{key}' and '{key}.1' are identical.")
    else:
        print(f"Columns '{key}' and '{key}.1' contain different values.")

Columns 'Unique Facility ID' and 'Unique Facility ID.1' contain different values.
Columns 'Status' and 'Status.1' contain different values.
Columns 'Name' and 'Name.1' contain different values.
Columns 'Type' and 'Type.1' contain different values.
Columns 'Plant Name' and 'Plant Name.1' contain different values.
Columns 'Current Operator' and 'Current Operator.1' contain different values.
Columns 'Co-Owner' and 'Co-Owner.1' contain different values.
Columns 'Constructed Date' and 'Constructed Date.1' contain different values.
Columns 'Latitude' and 'Latitude.1' contain different values.


In [71]:
duplicate_columns = geoscout_df.columns[geoscout_df.columns.duplicated()]
print(duplicate_columns)

Index([], dtype='object')


We do not have any duplicate columns in our geoscout_df. Although some columns have a '.1' suffix, they represent distinct data with differing row values compared to their corresponding columns.

Now let's look into some important columns with missing values that could be beneficial to dive into for our analysis.

* Longitude
* Latitude
* Name
* Type
* Plant Name
* Current Operator
* Location
* Sub Type
* Status
* Co-Owner

Carrying on the exact same preprocessing for the geoscout_df dataframe.

In [72]:
columns_to_check = ['Longitude', 'Latitude', 'Name', 'Type', 'Plant Name', 'Current Operator', 'Location', 'Sub Type', 'Status', 'Co-Owner']

for col in columns_to_check:
    print(f"Unique values in {col}:")
    print(geoscout_df[col].unique())
    print("\n")

Unique values in Longitude:
['112.15671 deg W' '113.07948 deg W' '113.79413 deg W' ...
 '122.68273 deg W' nan '121.15624 deg W']


Unique values in Latitude:
['52.14108 deg N' '52.31571 deg N' '49.30742 deg N' ... '59.20412 deg N'
 nan '56.57191 deg N']


Unique values in Name:
['MAPLE GLEN 15-36-036-16W4' 'KEYERA NEVIS GAS PLANT 16-33-38-22W4'
 '* GULF PINCHER CREEK' ...
 'NORTHRIVER OPSGP FORT NELSON NORTH B-047-I/094-P-04 001'
 'NORTHRIVER OPSGP DAWSONCREEK 11-26-078-17 003'
 'ALTAGAS HOLDINGS NORTH PINE 06-29-087-20 001']


Unique values in Type:
['GP' 'NF']


Unique values in Plant Name:
['Maple Glen' 'Nevis' 'Pincher Creek' 'Rimbey' 'Gilby' 'Mitsue' 'Princess'
 'Sedalia' 'Campbell' 'Innisfail' 'Makepeace' 'Carbon' 'Wardlow'
 'Cometra Wood River' 'Cessford' 'Provost' 'Buck Creek' 'Carstairs'
 'Garrington' 'Leduc Woodbend' 'Boundary Lake South' 'Normandville'
 'Golden Spike' 'Redwater' 'Ferrybank' 'Turner Valley' 'Valhalla'
 'Worsley' 'Three Hills Creek' 'Windfall' 'Icg Acheson' 'C

In [73]:
for col in columns_to_check:
    missing_count = geoscout_df[col].isnull().sum()
    print(f"Missing values in {col}: {missing_count}")

Missing values in Longitude: 2
Missing values in Latitude: 2
Missing values in Name: 135
Missing values in Type: 0
Missing values in Plant Name: 1186
Missing values in Current Operator: 133
Missing values in Location: 2
Missing values in Sub Type: 10769
Missing values in Status: 0
Missing values in Co-Owner: 6578


In [74]:
geoscout_df['Longitude'] = geoscout_df['Longitude'].str.extract(r'([\-+]?\d*\.\d+)').astype(float)
geoscout_df['Latitude'] = geoscout_df['Latitude'].str.extract(r'([\-+]?\d*\.\d+)').astype(float)

geoscout_df[['Longitude', 'Latitude']]

,Longitude,Latitude
0,112.15671,52.14108
1,112.15671,52.14108
2,112.15671,52.14108
3,112.15671,52.14108
4,112.15671,52.14108
...,...,...
223625,121.58278,59.20412
223626,120.52519,55.79045
223627,NaN,NaN
223628,NaN,NaN


In [75]:
geoscout_df['interest'] = geoscout_df.apply(
    lambda row: extract_interest(row['Co-Owner'], row['Current Operator']), axis=1
)

geoscout_df['interest'].unique()

array([  0. , 100. ,  98.8,  90. ,  80.2,  50. ,  20. ,  52. ,  72.8,
        53.3,  62. ,  79.8,  46.7,  87.3,  95. ,  98. ,  83.5,  51. ,
        40. ,  55. ,  99.8,  59. ,  90.9,  52.2,  75. ,  56.6,  45.8,
        96.1,  89.3,  85. ,  94.2,  82.2,  98.3,  75.9,  26. ,  82.8,
        38.9,  88.3,  80. ,  78.3,  74.9,  54. ,  70.8,  75.5,  63. ,
        43. ,  72.3,  65. ,  45. ,  45.6,  73.9,  61.4,  60. ,  89. ,
        62.9,  94.5,  65.7,  78. ,  75.6,  10.4,  19.1,  63.4,  83.3,
        37.2,  54.7,  86.5,  98.4,  84. ,  53. ,  83.1,  99.3,  93.9,
        69.5,  68. ,  99. ,  38.8,  55.1,  69.9,  42. ,  56. ,  31.8,
        60.1,  96. ,  88.7,  70. ,  74. ,  94. ,  54.6,  86.4,  62.7,
        72. ,  85.4,  76.7,  62.5,  78.2,  81. ,  34.9,  76.9,  44. ,
        64.2,  87.5,  76.8,  73. ,  38.5,  36.5,  66.7,  57. ,  88.2,
        87. ,  26.6,  41.4,  89.5,  69.7,  83. ])

In [76]:
geoscout_df['Plant Name'] = geoscout_df['Plant Name'].str.strip()
geoscout_df['Current Operator'] = geoscout_df['Current Operator'].str.strip()
geoscout_df['Status'] = geoscout_df['Status'].str.strip()
geoscout_df['Name'] = geoscout_df['Name'].str.strip()

In [77]:
name_mapping = geoscout_df.dropna(subset=['Name'])[['Unique Facility ID', 'Name']].set_index('Unique Facility ID')['Name'].to_dict()

geoscout_df['Name'] = geoscout_df.apply(
    lambda row: name_mapping.get(row['Unique Facility ID'], row['Name']) if pd.isna(row['Name']) else row['Name'],
    axis=1
)

geoscout_df['Name'].isnull().sum()

135

In [78]:
geoscout_df['Plant Name'].isnull().sum()

1186

In [79]:
for index, row in geoscout_df[geoscout_df['Plant Name'].isna() | geoscout_df['Name'].isna()].iterrows():
    missing_lat = row['Latitude']
    missing_lon = row['Longitude']

    match_row = geoscout_df[
        (geoscout_df['Latitude'] == missing_lat) & (geoscout_df['Longitude'] == missing_lon) &
        (geoscout_df['Plant Name'].notna()) & (geoscout_df['Name'].notna())
    ]

    if not match_row.empty:
        geoscout_df.at[index, 'Plant Name'] = match_row['Plant Name'].values[0]
        geoscout_df.at[index, 'Name'] = match_row['Name'].values[0]

In [80]:
print("Missing values in 'Plant Name':", geoscout_df['Plant Name'].isna().sum())
print("Missing values in 'Name':", geoscout_df['Name'].isna().sum())

Missing values in 'Plant Name': 783
Missing values in 'Name': 79


In [81]:
geoscout_df['Type'].nunique()

2

In [82]:
geoscout_df['Type']=='NF'

,Type
0,False
1,False
2,False
3,False
4,False
...,...
223625,False
223626,False
223627,False
223628,False


We had only only a single record for the other plant type available in the geoscout dataframe. Since the facility_df covers a longer time horizon with the same variables, we opted to use that dataframe.

In [83]:
geoscout_df.to_excel('/content/drive/My Drive/HTC_Case/geoscout_final_cleaned.xlsx', index=False)

Examining the emissions_df dataframe

In [84]:
emissions_df.head()

,GHGRP ID No. / No d'identification du PDGES,Reference Year / Année de référence,Facility Name / Nom de l'installation,Facility Location / Emplacement de l'installation,Facility City or District or Municipality / Ville ou District ou Municipalité de l'installation,Facility Province or Territory / Province ou territoire de l'installation,Facility Postal Code / Code postal de l'installation,Latitude,Longitude,Facility NPRI ID / Numéro d'identification de l'INRP,...,C6F14 (tonnes CO2e / tonnes éq. CO2),PFC Total (tonnes CO2e / tonnes éq. CO2),SF6 (tonnes),SF6 (tonnes CO2e / tonnes éq. CO2),Total Emissions (tonnes CO2e) / Émissions totales (tonnes éq. CO2),GHGRP Quantification Requirements / Exigences de quantification du PDGES,Emission Factors / Coefficients d'émission,Engineering Estimates / Estimations techniques,Mass Balance / Bilan massique,Monitoring or Direct Measurement / Surveillance ou mesure directe
0,G12721,2022,Lieu d'enfouissement technique de Rivière-du-Loup,1707 du Patrimoine Route,Cacouna,Quebec,G0L 1G0,47.9748,-69.43849,0.0,...,0,0.0,0.0,0.0,2.702042e+04,Not applicable / Sans Objet,Applicable,Applicable,Not applicable / Sans Objet,Not applicable / Sans Objet
1,G10001,2022,Division Alma,1100 Melançon Street,Alma,Quebec,G8B 5W2,48.5650,-71.65556,0.0,...,0,0.0,0.0,0.0,3.907723e+04,Applicable,Not applicable / Sans Objet,Not applicable / Sans Objet,Not applicable / Sans Objet,Not applicable / Sans Objet
2,G10003,2022,"Foothills Pipeline, Alberta",NaN,AIRDRIE,Alberta,T4A 2G7,NaN,NaN,0.0,...,0,0.0,0.0,0.0,3.526550e+05,Not applicable / Sans Objet,Applicable,Applicable,Not applicable / Sans Objet,Applicable
3,G10004,2022,Kingston CoGen,5146 Taylor-Kidd Boulevard,Bath,Ontario,K0H 1G0,44.2095,-76.72460,5765.0,...,0,0.0,0.0,0.0,1.011503e+03,Applicable,Not applicable / Sans Objet,Not applicable / Sans Objet,Not applicable / Sans Objet,Not applicable / Sans Objet
4,G10006,2022,Redwater Fertilizer Operations,56225 SH643,Sturgeon County,Alberta,T0A 2W0,53.8420,-113.09300,2134.0,...,0,0.0,0.0,0.0,1.226405e+06,Applicable,Applicable,Applicable,Applicable,Not applicable / Sans Objet


Shape of emissions_df dataframe

In [85]:
emissions_df.shape

(16823, 81)

All columns in the emissions_df dataframe

In [86]:
for col in emissions_df.columns:
    print(col)

GHGRP ID No. / No d'identification du PDGES
Reference Year / Année de référence
Facility Name / Nom de l'installation
Facility Location / Emplacement de l'installation
Facility City or District or Municipality / Ville ou District ou Municipalité de l'installation
Facility Province or Territory / Province ou territoire de l'installation
Facility Postal Code / Code postal de l'installation
Latitude
Longitude
Facility NPRI ID / Numéro d'identification de l'INRP
Facility NAICS Code / Code SCIAN de l'installation
English Facility NAICS Code Description / Description du code SCIAN de l'installation en anglais
French Facility NAICS Code Description / Description du code SCIAN de l'installation en français
Reporting Company Legal Name / Dénomination sociale de la société déclarante
Reporting Company Trade Name / Nom commercial de la société déclarante
Reporting Company Business Number / Numéro d'entreprise de la société déclarante
DUNS Number / Numéro DUNS
Public Contact Name / Nom du responsa

Checking for missing values in the emissions_df dataframe

In [87]:
emissions_df.isnull().sum()

,0
GHGRP ID No. / No d'identification du PDGES,0
Reference Year / Année de référence,0
Facility Name / Nom de l'installation,0
Facility Location / Emplacement de l'installation,4738
Facility City or District or Municipality / Ville ou District ou Municipalité de l'installation,1678
...,...
GHGRP Quantification Requirements / Exigences de quantification du PDGES,43
Emission Factors / Coefficients d'émission,43
Engineering Estimates / Estimations techniques,43
Mass Balance / Bilan massique,43


In [88]:
missing_counts = emissions_df.isnull().sum()
fully_missing_cols = missing_counts[missing_counts == emissions_df.shape[0]]

for col in fully_missing_cols.index:
    print(col)

No columns with all the rows missing.

Renaming columns to english only

In [89]:
emissions_df.columns = emissions_df.columns.str.split(' /').str[0]

In [90]:
emissions_df.columns

Index(['GHGRP ID No.', 'Reference Year', 'Facility Name', 'Facility Location',
       'Facility City or District or Municipality',
       'Facility Province or Territory', 'Facility Postal Code', 'Latitude',
       'Longitude', 'Facility NPRI ID', 'Facility NAICS Code',
       'English Facility NAICS Code Description',
       'French Facility NAICS Code Description',
       'Reporting Company Legal Name', 'Reporting Company Trade Name',
       'Reporting Company Business Number', 'DUNS Number',
       'Public Contact Name', 'Public Contact Position',
       'Public Contact Telephone', 'Public Contact Extension',
       'Public Contact Email', 'Public Contact Mailing Address',
       'Public Contact City or District or Municipality',
       'Public Contact Province or Territory', 'Public Contact Postal Code',
       'CO2 (tonnes)', 'CH4 (tonnes)', 'CH4 (tonnes CO2e', 'N2O (tonnes)',
       'N2O (tonnes CO2e', 'HFC-23 (tonnes)', 'HFC-23 (tonnes CO2e',
       'HFC-32 (tonnes)', 'HFC-32 (t

Removing an additional column "French Facility NAICS Code Description"

In [91]:
emissions_df = emissions_df.drop(columns=['French Facility NAICS Code Description'])

In [92]:
emissions_df.to_excel('/content/drive/My Drive/HTC_Case/emissions_final_cleaned.xlsx', index=False)